In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
from sklearn.metrics import mean_squared_log_error
# --- RMSLE function
def rmsle(y_true, y_pred):
    y_pred = np.maximum(0, y_pred)
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [3]:
train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv', parse_dates=['date'])
test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv', parse_dates=['date'])
transactions = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv', parse_dates=['date'])
oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv', parse_dates=['date'])
holidays = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv', parse_dates=['date'])
stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')

In [4]:
train

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [5]:
# merging all the files
def merge_dataset(df):
    df = df.merge(stores, how = 'left', on = 'store_nbr')
    oil['dcoilwtico'] = oil['dcoilwtico'].interpolate(method = 'linear')
    oil['dcoilwtico'] = oil['dcoilwtico'].bfill()
    df = df.merge(oil, how = 'left', on = 'date')
    #df = df.merge(transactions, how = 'left', on = ['date','store_nbr'])
    hol = holidays.copy()
    hol = hol.drop(hol[hol['transferred'] == 'True'].index)
    hol = holidays.copy()
    hol = hol.drop(hol[hol['transferred'] == 'True'].index)
    hol = hol.drop(columns = ['transferred'])
    hol['isholiday'] = 1
    lhol = hol.copy()
    lhol = lhol.drop(lhol[lhol['locale']!= 'Local'].index)
    lhol.rename(columns = {'locale_name':'city'},inplace = True)
    rhol = hol.copy()
    rhol = rhol.drop(rhol[rhol['locale']!= 'Regional'].index)
    rhol.rename(columns = {'locale_name':'state'},inplace = True)
    nhol = hol.copy()
    nhol = nhol.drop(nhol[nhol['locale']!= 'National'].index)
    df = df.merge(lhol[['date','city','isholiday']], how = 'left', on = ['date','city'])
    df = df.merge(rhol[['date','state','isholiday']], how = 'left', on = ['date','state'])
    df = df.merge(nhol[['date','isholiday']], how = 'left', on = 'date')
    df['isHoliday'] = df['isholiday_x']+df['isholiday_y']+df['isholiday']
    df = df.drop(columns = ['isholiday_x','isholiday_y','isholiday'])
    df['isHoliday'] = df['isHoliday'].fillna(0).astype('Int8')
    df['Is_Weekend'] = [1 if x >= 5 else 0 for x in df['date'].dt.dayofweek]
    df = df.drop(columns = ['date','store_nbr',])
    df = pd.get_dummies(df, columns=['family','city','state','type',])
    df = df.bfill()
    return df

In [6]:
train = merge_dataset(train)
test = merge_dataset(test)
train = train.drop(columns = ['id'])

In [7]:
train

,sales,onpromotion,cluster,dcoilwtico,isHoliday,Is_Weekend,family_AUTOMOTIVE,family_BABY CARE,family_BEAUTY,family_BEVERAGES,...,state_Pastaza,state_Pichincha,state_Santa Elena,state_Santo Domingo de los Tsachilas,state_Tungurahua,type_A,type_B,type_C,type_D,type_E
0,0.000,0,13,93.14,0,0,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
1,0.000,0,13,93.14,0,0,False,True,False,False,...,False,True,False,False,False,False,False,False,True,False
2,0.000,0,13,93.14,0,0,False,False,True,False,...,False,True,False,False,False,False,False,False,True,False
3,0.000,0,13,93.14,0,0,False,False,False,True,...,False,True,False,False,False,False,False,False,True,False
4,0.000,0,13,93.14,0,0,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3008275,438.133,0,6,47.57,0,0,False,False,False,False,...,False,True,False,False,False,False,True,False,False,False
3008276,154.553,1,6,47.57,0,0,False,False,False,False,...,False,True,False,False,False,False,True,False,False,False
3008277,2419.729,148,6,47.57,0,0,False,False,False,False,...,False,True,False,False,False,False,True,False,False,False
3008278,121.000,8,6,47.57,0,0,False,False,False,False,...,False,True,False,False,False,False,True,False,False,False


In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

In [9]:
#train validation split
x_train = train.drop(columns = ['sales'])
y_train = train['sales']
xt,xv,yt,yv = train_test_split(x_train,y_train)

In [10]:
# Initialize Bagging Regressor
model = BaggingRegressor(
    estimator=DecisionTreeRegressor(max_depth=3),  
    n_estimators=100,          # Number of base models
    random_state=42          
)

# Fit to your data
model.fit(xt, yt)

#prediction 
y_hat = model.predict(xv)

#validation
rmsle(yv, y_hat)


3.372267808302129

In [11]:
test_ = test.drop(columns = ['id'])
y_pred = model.predict(test_)

In [12]:
result = pd.DataFrame()
result['id'] = test['id']
result['sales'] = y_pred

In [13]:
result

,id,sales
0,3000888,143.441253
1,3000889,143.441253
2,3000890,143.441253
3,3000891,1915.450706
4,3000892,143.441253
...,...,...
28507,3029395,143.441253
28508,3029396,143.441253
28509,3029397,1348.561350
28510,3029398,143.441253


In [14]:
result.to_csv("submission.csv", index=False)